In [1]:
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.affinity import scale

In [2]:
import tropycal.realtime as realtime
import geojson
import os
import math
import numpy as np
import timeit

In [3]:
# Initialize the real-time object
# realtime_obj = realtime.Realtime()

In [4]:
# List of basins to check for active hurricanes (North Atlantic, East Pacific, and Central Pacific)
basins = ['north_atlantic', 'east_pacific', 'central_pacific']

In [5]:
# Function to check if a value is valid (i.e., not NaN or infinity)
def is_valid_value(value):
    return value is not None and not math.isnan(value) and not math.isinf(value)

In [6]:
# Function to create an envelope around coordinates by adding a buffer
def create_envelope(coords, buffer=0.3):
    buffered_coords = []
    for lon, lat in coords:
        buffered_coords.append((lon + buffer, lat + buffer))
        buffered_coords.append((lon - buffer, lat + buffer))
        buffered_coords.append((lon + buffer, lat - buffer))
        buffered_coords.append((lon - buffer, lat - buffer))
    
    # Get unique coordinates and sort them
    unique_coords = list(set(buffered_coords))
    unique_coords.sort(key=lambda x: (x[0], x[1]))
    
    # Create the envelope polygon
    multipoint = MultiPoint(unique_coords)
    convex_hull = multipoint.convex_hull
    exterior_coords = list(convex_hull.exterior.coords)
    
    return exterior_coords

In [7]:
# Function to fetch active storm data (ignoring invests) and convert it to GeoJSON format
def get_active_storms_geojson():
    features = []
    
    # Loop through each basin
    for basin in basins:
        try:
            # List all active storms in the basin
            active_storms = realtime_obj.list_active_storms(basin=basin)
            
            for storm_name in active_storms:
                try:
                    # Retrieve full storm data
                    storm = realtime_obj.get_storm(storm_name)
                    
                    # Ignore Invest storms by checking the storm's classification
                    if storm.invest:
                        print(f"Ignoring Invest: {storm_name}")
                        continue  # Skip Invest storms
                    
                    # Get the latest forecast track (real-time forecast data)
                    forecast_data = storm.get_forecast_realtime()
                    
                    # Create GeoJSON polygons for the forecast area
                    coordinates = []
                    for i in range(len(forecast_data['lat'])):
                        lat = forecast_data['lat'][i]
                        lon = forecast_data['lon'][i]
                        
                        if is_valid_value(lat) and is_valid_value(lon):
                            coordinates.append((lon, lat))
                    
                    # Create an envelope around the coordinates
                    if len(coordinates) > 2:
                        envelope_coords = create_envelope(coordinates)
                        if envelope_coords:
                            # Create GeoJSON Polygon
                            polygon = geojson.Polygon([envelope_coords])
                            
                            # Create GeoJSON Feature with valid properties
                            feature = geojson.Feature(
                                geometry=polygon,
                                properties={
                                    'storm_name': storm.name,
                                    'storm_id': storm.id,
                                    'basin': basin,
                                    'storm_type': forecast_data['type'][0] if len(forecast_data['type']) > 0 else None
                                }
                            )
                            features.append(feature)
                
                except Exception as e:
                    print(f"Error retrieving data for storm {storm_name}: {e}")
                    
        except Exception as e:
            print(f"Error retrieving storms for basin {basin}: {e}")
    
    # Return the GeoJSON FeatureCollection
    return geojson.FeatureCollection(features)

In [8]:
# Define the function to test time required to execute
def time_required_to_execute():
    # Initialize the real-time object
    realtime_obj = realtime.Realtime()

    # # Fetch the active storms and convert to GeoJSON
    geojson_data = get_active_storms_geojson()

In [9]:
# Specify the path where you want to save the GeoJSON file
save_path = "C:/Downloads/active_storms.geojson"  # Replace with your path

In [10]:
# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [11]:
# # Fetch the active storms and convert to GeoJSON
# geojson_data = get_active_storms_geojson()

In [12]:
# Measure the time required to execute
elapsed_time = timeit.timeit(time_required_to_execute, number=1)
print(f"Time taken to execute the function: {elapsed_time:.2f} seconds")

--> Starting to read in current storm data
--> Completed reading in current storm data (22.22 seconds)
Error retrieving storms for basin north_atlantic: name 'realtime_obj' is not defined
Error retrieving storms for basin east_pacific: name 'realtime_obj' is not defined
Error retrieving storms for basin central_pacific: name 'realtime_obj' is not defined
Time taken to execute the function: 24.74 seconds


In [13]:
# Save the GeoJSON data to the specified path
with open(save_path, 'w') as f:
    geojson.dump(geojson_data, f, indent=2)

print(f"GeoJSON file with active storm data has been created at {save_path}.")

NameError: name 'geojson_data' is not defined